In [1]:
%pip install bitshuffle
%pip install Pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
#from ultralytics import YOLO
import time
import os
import random
import psutil
import scipy.ndimage
from PIL import Image
from PIL import ImageDraw
from pathlib import Path
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
import cv2
from skimage.feature import hog
%matplotlib inline

In [4]:
df = pd.read_csv('/home/cgchoza/galaxies/complete_cadences_catalog.csv')
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
36553,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36554,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36555,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36556,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [6]:
df['.h5 path'] = df['.h5 path'].str.replace('0000.h5', '0002.h5', regex=False)
df = df.drop_duplicates(subset='.h5 path', keep='first').reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
30309,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30310,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30311,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30312,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [7]:
df = df[~df['.h5 path'].str.contains('spliced')].reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
1,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
2,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
3,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
4,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
...,...,...,...,...,...,...,...,...
29341,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29342,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29343,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29344,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df = df.drop(index = 17546)
df = df.sample(n=1000, random_state=42).reset_index(drop=True)

In [13]:
# def process_scan_for_yolo(index, h5_path, base_dir,
#                           coarse_channel_width=128,
#                           factor=7,
#                           dilation_size=(10,300),
#                           class_id=0):
#     """
#     For each coarse channel:
#       - generate a PNG of the waterfall slice,
#       - produce a .txt file of YOLO bboxes for each connected hit region.
#     """
#     out_dir = Path(out_dir)
#     img_dir = out_dir / "images"
#     img_dir.mkdir(parents=True, exist_ok=True)

#     stem = Path(h5_path).stem

#     # 1) load & log-scale
#     fb   = bl.Waterfall(h5_path, load_data=True)
#     data = 10 * np.log10(fb.data.squeeze())   # shape (ntime, nfreq)
#     nt, nf = data.shape
#     n_coarse = nf // coarse_channel_width

#     # 2) global threshold
#     m, s = np.median(data), np.std(data)
#     binary = data > (m + factor * s)

#     # 3) dilate to merge neighboring hits
#     binary = scipy.ndimage.maximum_filter(binary, size=dilation_size)

#     # 4) make directories
#     img_tr = Path(base_dir)/"train"/"images"
#     lbl_tr = Path(base_dir)/"train"/"labels"
#     img_vl = Path(base_dir)/"val"/"images"
#     lbl_vl = Path(base_dir)/"val"/"labels"
#     for d in (img_tr, lbl_tr, img_vl, lbl_vl):
#         d.mkdir(parents=True, exist_ok=True)

#     stem = Path(h5_path).stem

#     # 4) per-block: find components, save image + YOLO .txt
#     for i in range(n_coarse):
#         f0, f1 = i * coarse_channel_width, (i + 1) * coarse_channel_width
#         block = data[:, f0:f1]
#         mask  = binary[:, f0:f1]

#         # connected components
#         labeled, n_objs = scipy.ndimage.label(mask)

#         # 1) extract the slice
#         block = data[:, f0:f1]   # shape: (ntime, coarse_channel_width)

#         # 2) threshold & find connected components as before
#         mask  = binary[:, f0:f1]
#         labeled, n_objs = scipy.ndimage.label(mask)

#         # 3) convert block to 8-bit RGB JPEG
#         #    normalize just this block so it fills 0–255
#         arr8 = (255 * (block - block.min()) / block.ptp()).astype(np.uint8)
#         rgb  = np.stack([arr8]*3, axis=-1)        # (H, W, 3)
#         img = Image.fromarray(rgb)
#         img_name = f'img_{index:05d}.png'
#         out_path = os.path.join(img_dir, img_name)
#         img.save(out_path)


#     # 4) write the matching .txt with the same basename
#     txt_name = img_name.replace(".png", ".txt")
#     with open(img_dir / txt_name, "w") as ftxt:
#         for lab in range(1, n_objs + 1):
#             ys, xs = np.where(labeled == lab)
#             y_min, y_max = ys.min(), ys.max()
#             x_min, x_max = xs.min(), xs.max()
#             # normalized YOLO coords
#             x_c = ((x_min + x_max) / 2) / coarse_channel_width
#             y_c = ((y_min + y_max) / 2) / nt
#             w   = (x_max - x_min) / coarse_channel_width
#             h   = (y_max - y_min) / nt
#             ftxt.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")


#         # write YOLO .txt (one line per blob)
#         txt_fn = img_dir / f"{stem}_chan{i:03d}.txt"
#         with open(txt_fn, "w") as ftxt:
#             for lab in range(1, n_objs + 1):
#                 ys, xs = np.where(labeled == lab)
#                 y_min, y_max = ys.min(), ys.max()
#                 x_min, x_max = xs.min(), xs.max()

#                 # normalized YOLO coords
#                 x_c = ((x_min + x_max) / 2) / coarse_channel_width
#                 y_c = ((y_min + y_max) / 2) / nt
#                 w   = (x_max - x_min) / coarse_channel_width
#                 h   = (y_max - y_min) / nt

#                 ftxt.write(f"0 {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")


In [ ]:
fb = bl.Waterfall(df['.h5 path'].iloc[-1], load_data=True)
fb.info()


--- File Info ---
DIMENSION_LABELS : [b'time' b'feed_id' b'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :               10501.46484375 MHz
            foff :         -0.00286102294921875 MHz
           ibeam :                               -1
      machine_id :                               20
          nbeams :                                1
           nbits :                               32
          nchans :                            65536
            nifs :                                1
     rawdatafile : guppi_59411_54386_HIP30264_0094.0000.raw
     source_name :                         HIP30264
         src_dej :                     -8:26:53.228
         src_raj :                      6:21:58.451
    telescope_id :                                6
           tsamp :                1.073741823999999
   tstart (ISOT) :          2021-07-16T15:06:26.000
    tstart (MJD) :                59

In [ ]:
# from collections import namedtuple

# # Define a named tuple for bounding boxes
# BoundingBox = namedtuple("BoundingBox", ["x_c", "y_c", "w", "h", "area", "iou", "class_id"])

# def calculate_iou(box1, box2):
#     """
#     Calculate Intersection over Union (IoU) between two bounding boxes.
#     """
#     x1_min = box1.x_c - box1.w / 2
#     x1_max = box1.x_c + box1.w / 2
#     y1_min = box1.y_c - box1.h / 2
#     y1_max = box1.y_c + box1.h / 2

#     x2_min = box2.x_c - box2.w / 2
#     x2_max = box2.x_c + box2.w / 2
#     y2_min = box2.y_c - box2.h / 2
#     y2_max = box2.y_c + box2.h / 2

#     # Calculate intersection
#     inter_x_min = max(x1_min, x2_min)
#     inter_x_max = min(x1_max, x2_max)
#     inter_y_min = max(y1_min, y2_min)
#     inter_y_max = min(y1_max, y2_max)

#     if inter_x_min < inter_x_max and inter_y_min < inter_y_max:
#         intersection = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
#     else:
#         intersection = 0

#     # Calculate union
#     area1 = box1.w * box1.h
#     area2 = box2.w * box2.h
#     union = area1 + area2 - intersection

#     return intersection / union if union > 0 else 0

# def non_maximum_suppression(boxes, iou_threshold=0.5):
#     """
#     Perform Non-Maximum Suppression (NMS) to remove overlapping boxes.
#     """
#     if not boxes:
#         return []

#     # Sort boxes by area (or confidence if available)
#     boxes = sorted(boxes, key=lambda b: b.area, reverse=True)

#     selected_boxes = []
#     while boxes:
#         # Select the box with the largest area
#         current_box = boxes.pop(0)
#         selected_boxes.append(current_box)

#         # Remove boxes with IoU > threshold
#         boxes = [
#             box for box in boxes
#             if calculate_iou(current_box, box) <= iou_threshold
#         ]

#     return selected_boxes

In [ ]:
# 4) SET UP LOGGING
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

def build_tasks(df):
    """
    Build a flat list of (h5_path, channel_idx) from your DataFrame.
    Uses df['nchans'] if present, otherwise reads header to find nfreq.
    """
    tasks = []
    for _, row in df.iterrows():
        h5 = row[".h5 path"]
        fb = bl.Waterfall(h5, load_data=False)
        nfreq = fb.header.get("nchans")
        nfpc = fb.header.get("nfpc", 1024)  # default to 1024 if not set
        n_coarse = nfreq // nfpc

        for ch in range(n_coarse):
            tasks.append((h5, ch))
    return tasks

def split_tasks(tasks, train_frac=0.8, seed=42):
    """
    Shuffle & split the flat task list into train vs. val sets.
    Returns two sets of (h5_path, channel_idx).
    """
    random.seed(seed)
    shuffled = tasks.copy()
    random.shuffle(shuffled)
    cut = int(train_frac * len(shuffled))
    train = set(shuffled[:cut])
    val   = set(shuffled[cut:])
    return train, val

def process_file(job):
    """
    job is a tuple:
      (h5_path, [ch_idx,...], [global_idx,...],
       base_dir, coarse_channel_width, factor, dilation_size,
       class_id, train_set, val_set)
    """
    (h5_path,
     channels,
     global_indices,
     base_dir,
     factor,
     dilation,
     class_id,
     train_set,
     val_set,
     pad_width) = job

    # 1) load the waterfall once
    fb   = bl.Waterfall(h5_path, load_data=True)
    data = 10 * np.log10(fb.data.squeeze())   # shape (ntime, nfreq)
    nt, nf = data.shape

    #  # 2) threshold + dilation
    # M, S = np.median(data), np.std(data)
    # binary = data > (M + factor * S)
    # binary = scipy.ndimage.maximum_filter(binary, size=dilation)

    # 3) process each channel
    for ch_idx, gidx in zip(channels, global_indices):
        cw = fb.header.get("nfpc", 1024)  # default to 1024 if not set
        f0, f1 = ch_idx*cw, (ch_idx+1)*cw
        block  = data[:, f0:f1]

        # decide train vs val
        subset = "train" if (h5_path, ch_idx) in train_set else "val"

        # make sure all dirs exist
        img_dir = Path(base_dir)/subset/"images"
        lbl_dir = Path(base_dir)/subset/"labels"
        vis_dir = Path(base_dir)/"visualization"/subset
        for d in (img_dir, lbl_dir, vis_dir):
            d.mkdir(parents=True, exist_ok=True)

        # Get rid of rolloff
        n_cols = block.shape[1]
        low = int(0.1 * n_cols)
        high = int(0.9 * n_cols)
        block_middle80 = block[:, low:high]

        # Remove vertical line artifact
        rows, cols = block_middle80.shape
        vert_means = block_middle80.mean(axis=0)
        center = np.argmax(vert_means)
        left_col = center - 1
        right_col = center + 1
        if left_col >= 0 and right_col < cols:
            block_middle80[:, center] = (block_middle80[:, left_col] + block_middle80[:, right_col]) / 2
        elif left_col >= 0:
            block_middle80[:, center] = block_middle80[:, left_col]
        elif right_col < cols:
            block_middle80[:, center] = block_middle80[:, right_col]

        # ====== NOW threshold and dilate on the CLEANED block ======
        #import scipy.ndimage

        # Normalize for visualization + processing
        arr8 = (255 * (block_middle80 - block_middle80.min()) / block_middle80.ptp()).astype(np.uint8)
        gray = arr8
        h, w = gray.shape

        # --- Compute HOG magnitude image ---
        hog_feats, hog_image = hog(
            gray,
            orientations=9,
            pixels_per_cell=(8, 8),
            cells_per_block=(2, 2),
            visualize=True,
            feature_vector=False
        )
        # hog_image is float64; normalize to [0,255] uint8
        hog_uint8 = cv2.normalize(hog_image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        # --- Threshold & clean the HOG map ---
        # Choose threshold by inspection; e.g. 30 out of 255
        _, bin_mask = cv2.threshold(hog_uint8, 30, 255, cv2.THRESH_BINARY)
        # Remove small speckles
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        mask_clean = cv2.morphologyEx(bin_mask, cv2.MORPH_OPEN, kernel, iterations=1)
        # (Optional) Close gaps in long thin edges
        mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_CLOSE, kernel, iterations=1)


        # Calculate frequency range of this coarse channel
        f_start = fb.header['fch1'] + f0 * fb.header['foff']
        f_stop  = fb.header['fch1'] + (f1 - 1) * fb.header['foff']

        # Prepare and save PNG
        arr8 = (255 * (block_middle80 - block_middle80.min()) / block_middle80.ptp()).astype(np.uint8)
        rgb = np.stack([arr8] * 3, axis=-1)
        img = Image.fromarray(rgb)
        fn = f"img_{gidx:0{pad_width}d}_f_{f_start:.4f}_{f_stop:.4f}.png"
        img_path = img_dir / fn
        img.save(img_path)

        # Save YOLO boxes
        txt_path = lbl_dir / fn.replace(".png", ".txt")
        h, w = gray.shape
        min_size = 3
        max_rel_size = 0.9

        contours, _ = cv2.findContours(
        mask_clean,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
        )

        slices = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < 20:           # skip tiny blobs
                continue
            x, y, w, h = cv2.boundingRect(cnt)
            # skip boxes too big
            if (w*h) / (w_img*h_img) > 0.9:
                continue
            slices.append((slice(y, y+h), slice(x, x+w)))
        logging.info(f"HOG → found {len(slices)} regions")

        all_boxes = []
        all_boxes = []
        for sl in slices:
            y1, y2 = sl[0].start, sl[0].stop
            x1, x2 = sl[1].start, sl[1].stop
            box_w = x2 - x1
            box_h = y2 - y1
            x_center = (x1 + box_w/2) / w_img
            y_center = (y1 + box_h/2) / h_img
            norm_w = box_w / w_img
            norm_h = box_h / h_img
            all_boxes.append((class_id, x_center, y_center, norm_w, norm_h))

        # Greedy suppression: remove boxes fully inside larger ones
        def is_inside(inner, outer):
            ix, iy, iw, ih = inner[:4]
            ox, oy, ow, oh = outer[:4]
            return (
                abs(ix - ox) < 0.5 * (ow - iw) and
                abs(iy - oy) < 0.5 * (oh - ih)
            )

        all_boxes.sort(key=lambda b: b[4], reverse=True)  # largest first
        final_boxes = []
        for box in all_boxes:
            if not any(is_inside(box, keep) for keep in final_boxes):
                final_boxes.append(box)

        # Write boxes to file
        with open(txt_path, "w") as ftxt:
            for x_c, y_c, w_n, h_n, _ in final_boxes:
                ftxt.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}\n")

        # If nothing survived, remove the .txt file
        if not final_boxes:
            if txt_path.exists():
                txt_path.unlink()
            with open(lbl_dir / 'empty_labels.csv', 'a') as f:
                f.write(f"{fn},{h5_path},{ch_idx}\n")



        # —————————————————————————————
        # 4) Draw boxes on the saved PNG
        # —————————————————————————————
        vis_img = Image.open(img_path).convert("RGB")
        draw    = ImageDraw.Draw(vis_img)
        w_img, h_img = vis_img.size


        if txt_path.exists():
            with open(txt_path) as ftxt:
                for line in ftxt:
                    cls, x_c, y_c, bw, bh = map(float, line.split())
                    xc = x_c * w_img
                    yc = y_c * h_img
                    bw_pix = bw * w_img
                    bh_pix = bh * h_img
                    x0 = int(xc - bw_pix/2)
                    y0 = int(yc - bh_pix/2)
                    x1 = int(xc + bw_pix/2)
                    y1 = int(yc + bh_pix/2)
                    draw.rectangle([x0, y0, x1, y1], outline="red", width=2)

        vis_img.save(vis_dir/fn)

        
if __name__ == "__main__":
    # === your settings ===
    BASE_DIR             = "/datax/scratch/jliang/dataset_hog"
    FACTOR               = 6
    DILATION_SIZE        = (30, 30)
    CLASS_ID             = 0
    TRAIN_FRAC           = 0.8
    SEED                 = 42
    NUM_WORKERS          = 8

    # --- assume you already have a DataFrame `df` with at least 'h5_path' and optionally 'nchans' ---
    # df = pd.read_csv(...)  # or however you built it

    # 1) Build & split tasks
    tasks = build_tasks(df)
    train_set, val_set = split_tasks(tasks, TRAIN_FRAC, SEED)

    # 2) compute zero-pad width from total images
    pad_width = len(str(len(tasks) - 1))

    # 3) regroup tasks by file to load each .h5 only once
    jobs = {}
    for gidx, (h5, ch) in enumerate(tasks):
        jobs.setdefault(h5, {"chs": [], "gidxs": []})
        jobs[h5]["chs"].append(ch)
        jobs[h5]["gidxs"].append(gidx)

    # 4) prepare job‐tuples
    job_list = []
    for h5, info in jobs.items():
        job_list.append((
            h5,
            info["chs"],
            info["gidxs"],
            BASE_DIR,
            FACTOR,
            DILATION_SIZE,
            CLASS_ID,
            train_set,
            val_set,
            pad_width
        ))

    # 5) run in parallel, one process per file
    # with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
    #     for _ in as_completed(exe.map(process_file, job_list)):
    #         passf

    with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
        futures = [exe.submit(process_file, job) for job in job_list]
        for future in as_completed(futures):
            result = future.result()

    logging.info("All done.")
